In [196]:
import twitter
import re
import os
import csv
from textblob import TextBlob
import pandas as pd
from datetime import datetime

In [77]:
api = twitter.Api(consumer_key = os.environ['Twitter_Ckey'], 
                  consumer_secret= os.environ['Twitter_Csecret'], 
                  access_token_key= os.environ['Twitter_Atoken'], 
                  access_token_secret= os.environ['Twitter_ATsecret'], sleep_on_rate_limit=True, tweet_mode='extended')

In [150]:
# Preprocess Tweets
def preProcessTweets(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet   

In [151]:
def getTweetsData(search_term, tweet_count, date):
    try:
        tweets = api.GetSearch(search_term, count=tweet_count, since = date) # date format is YYYY-MM-DD
        return [{"Text":preProcessTweets(status.full_text), "User":status.user.screen_name, "UserName":status.user.name, "CreatedAt": status.created_at} for status in tweets]
    except:
        print('Error occured')
        return None

In [152]:
def get_tweet_sentiment(tweet):
        '''
        Function to know tweet's sentiment
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(tweet)
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [192]:
def sentimentPercent(tweetD):
    if len(tweetD) >0:
        # picking positive tweets from tweets
        ptweets = [tweet['Text'] for tweet in tweetD if get_tweet_sentiment(tweet['Text']) == 'positive']
        # percentage of positive tweets
        print("Positive tweets percentage: {} %".format(round(100*len(ptweets)/len(tweetD),2)))
        # picking negative tweets from tweets
        ntweets = [tweet['Text'] for tweet in tweetD if get_tweet_sentiment(tweet['Text']) == 'negative']
        # percentage of negative tweets
        print("Negative tweets percentage: {} %".format(round(100*len(ntweets)/len(tweetD),2)))
        # picking neutral tweets from tweets
        nutweets = [tweet['Text'] for tweet in tweetD if get_tweet_sentiment(tweet['Text']) == 'neutral']
        # percentage of neutral tweets
        print("Neutral tweets percentage: {} %".format(round(100*len(nutweets)/len(tweetD),2)))
    else:
        print("No tweet data for sentiment analysis")

In [235]:
searchTerm = input('Enter your search term:')
tweetCount = input('How many tweets you want to fetch, enter the count:')
tweetSince = input('Enter the date in format: YYYY-MM-DD to fetch tweets from that date:')
tweetsData = getTweetsData(searchTerm, tweetCount, tweetSince)

Enter your search term:Bitcoin
How many tweets you want to fetch, enter the count:5
Enter the date in format: YYYY-MM-DD to fetch tweets from that date:2018-04-07


In [240]:
mdt = tweetsData[0]['User'].encode("utf-8")
mdtt = mdt.decode("utf-8")

In [202]:
# Open a csv file to store the Tweets Data
file_name = 'Sentiment_Analysis_of_{}_Tweets_About_{}.csv'.format(tweetCount, searchTerm)
with open(file_name, 'w', newline='') as csvFile:
    #Use csv Writer
    csvWriter = csv.DictWriter(f=csvFile, fieldnames=["TweetText", "User", "UserName", "CreatedAt", "Sentiment"])
    csvWriter.writeheader()
    for i in range(0,len(tweetsData)):
        senti = get_tweet_sentiment(tweetsData[i]['Text'])
        csvWriter.writerow({"TweetText":tweetsData[i]['Text'].encode("utf-8"), 
                             "User":tweetsData[i]['User'].encode("utf-8"), 
                             "UserName":tweetsData[i]['UserName'].encode("utf-8"), 
                             "CreatedAt":tweetsData[i]['CreatedAt'],"Sentiment":senti})

In [203]:
sentimentPercent(tweetsData)

Positive tweets percentage: 60.0 %
Negative tweets percentage: 10.0 %
Nutral tweets percentage: 30.0 %


In [195]:
import sqlite3

In [225]:
conn = sqlite3.connect('twitterData.db')

In [234]:
targettwitterTable = 'TwitterTable'
c = conn.cursor()
try:
    c.execute('CREATE TABLE '+ targettwitterTable +'(dbtweettext text, dbtwittersentiment text)')
except:
    print('TwitterTable already exists')

TwitterTable already exists


In [227]:
# update data in the table
for i in range(0,len(tweetsData)):
    c.execute('insert into TwitterTable values (?,?)', (tweetsData[i]['Text'].encode("utf-8"), 
                                                        get_tweet_sentiment(tweetsData[i]['Text'])))

In [205]:
# commit the db and close the connection
conn.commit()
c.close()

In [ ]:
## Fetch saved data

In [226]:
conn = sqlite3.connect('twitterData.db')
c = conn.cursor()

In [231]:
rslt = c.execute('Select * from TwitterTable')

In [230]:
for i in rslt:
    print(i)

In [224]:
c.close()